In [2]:
import tensorflow as tf
import numpy as np

In [17]:
char_arr = [c for c in  'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
char2num = {n: i for i, n in enumerate(char_arr)}
dic_len = len(char2num)
english = ['word', 'wood', 'game', 'girl', 'kiss', 'love']
korean = ['단어', '나무', '놀이', '소녀', '키스', '사랑']
seq_data = np.stack((english, korean), 1)

In [18]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [char2num[n] for n in seq[0]]
        output = [char2num[n] for n in ('S' + seq[1])]
        target = [char2num[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

In [19]:
input_batch, output_batch, target_batch = make_batch(seq_data)

In [20]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 500
n_class = n_input = dic_len

In [21]:
tf.reset_default_graph()
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [22]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

In [23]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state = enc_states, dtype=tf.float32)

In [24]:
model = tf.layers.dense(outputs, n_class, activation=None)

In [25]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

In [26]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [27]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    if epoch % 50 == 49:
        print('Epoch:', '%03d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
    
print('optimization finished!')

Epoch: 050 cost = 0.015707
Epoch: 100 cost = 0.003501
Epoch: 150 cost = 0.002658
Epoch: 200 cost = 0.000890
Epoch: 250 cost = 0.001440
Epoch: 300 cost = 0.000449
Epoch: 350 cost = 0.000430
Epoch: 400 cost = 0.000666
Epoch: 450 cost = 0.000151
Epoch: 500 cost = 0.000176
optimization finished!


In [30]:
def translate(word):
    seq_data = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    decoded = [char_arr[i] for i in result[0]]
    
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    
    return translated

In [31]:
print('===translation test===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('read ->', translate('read'))
print('game ->', translate('game'))

===translation test===
word -> 단어
wodr -> 단어
love -> 사랑
loev -> 사랑
read -> 소녀
game -> 놀이
